# Load Dependencies

In [1]:
# -*- coding: unicode -*-

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from matplotlib import pyplot
import gensim
import nltk
from sklearn import metrics
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import TensorBoard

Using TensorFlow backend.
C:\Users\insuf\AppData\Local\conda\conda\envs\python352\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [4]:
assert gensim.models.doc2vec.FAST_VERSION > -1

In [5]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\insuf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\insuf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13283795747212513169
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3129039257
locality {
  bus_id: 1
}
incarnation: 4024810860380551380
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0"
]


# Data Preprocessing

In [7]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [8]:
train = pd.read_csv('train_clean.csv')
train.comment_text = train.comment_text.astype(str)

In [9]:
list_classes = list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]

In [27]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
len(tokenizer.word_index)

223043

In [28]:
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [29]:
# list_sentences_train

In [30]:
# X_t

In [17]:
description = train.comment_text.values.tolist()
stop_words = set(nltk.corpus.stopwords.words('english'))


# Function to filter stop words from tokens for each sentence
def sentence_filter(sentence_token, stop_words):
    filtered = []
    for token in sentence_token:
        if not token in stop_words:
            filtered.append(token)
    return filtered


# Input description, tokenize each sentence and return token for each sentence without stopping words
def sentence_tokenizer(description, stop_words):
    value = []
    for sentence in description:
        sentence = sentence.lower()
        tokenizer_ = nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+')
        sentence_token = tokenizer_.tokenize(sentence)
        # sentence_token = nltk.word_tokenize(sentence)
        filtered_token = sentence_filter(sentence_token, stop_words)
        value.append(filtered_token)
    return value


# Tokenize the sentence
description_token = sentence_tokenizer(description, stop_words)

In [18]:
len(description_token)

159571

In [46]:
# Skip gram model with minimum word count = 10 and output vector of length 300
# model_sg = gensim.models.Word2Vec(description_token, window=2, min_count=10, size=300, workers=4, seed=123, iter=10)

# CBOW(continuous bag of words) model
model_cbow = gensim.models.Word2Vec(description_token, window=2, min_count=10, size=300, workers=4, seed=123, iter=10)

In [20]:
model_sg.save('model_sg')

In [47]:
model_cbow.save('model_cbow')

In [31]:
model_sg = gensim.models.Word2Vec.load('model_sg')

embed_size = 300
embeddings_index = dict()
for word in model_sg.wv.vocab:
    embeddings_index[word] = model_sg.wv[word]
print('Loaded %s word vectors.' % len(embeddings_index))

nb_words = min(max_features, len(tokenizer.word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

embeddedCount = 0
for word, i in tokenizer.word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        embeddedCount += 1
print('total embedded:',embeddedCount,'common words')

Loaded 25653 word vectors.
total embedded: 19861 common words


In [48]:
model_cbow = gensim.models.Word2Vec.load('model_cbow')

embed_size = 300
embeddings_index = dict()
for word in model_cbow.wv.vocab:
    embeddings_index[word] = model_cbow.wv[word]
print('Loaded %s word vectors.' % len(embeddings_index))

nb_words = min(max_features, len(tokenizer.word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

embeddedCount = 0
for word, i in tokenizer.word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        embeddedCount += 1
print('total embedded:',embeddedCount,'common words')

Loaded 25653 word vectors.
total embedded: 19861 common words


In [32]:
embedding_matrix.shape

(20000, 300)

In [54]:
model_cbow.wv.most_similar('fuck')

[('fucking', 0.5763519406318665),
 ('fuk', 0.5572361946105957),
 ('bitch', 0.5500202178955078),
 ('fuckin', 0.5281999707221985),
 ('fck', 0.5242219567298889),
 ('cocksucker', 0.5240716934204102),
 ('jerk', 0.517245352268219),
 ('fucker', 0.511810302734375),
 ('fuckwit', 0.5083379745483398),
 ('retards', 0.5072153806686401)]

In [33]:
def auc(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

In [34]:
from keras.models import Sequential

model = Sequential()
model.add(Embedding(max_features,embed_size,weights=[embedding_matrix],input_length=maxlen))
model.add(Bidirectional(LSTM(60,return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(6, activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[auc])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          6000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 120)          173280    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                6050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 306       
Total params: 6,179,636
Trainable params: 6,179,636
Non-trainable params: 0
_________________________________________________________________


In [18]:
Y = []
for item in y.tolist():
    for value in item:
        Y.append(value)

In [19]:
from collections import Counter

def get_class_weights(y, smooth_factor=0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    return {cls: float(majority / count) for cls, count in counter.items()}

get_class_weights(Y, smooth_factor=0.1)

{0: 1.0, 1: 7.967913497755453}

In [35]:
tensorBoardCB = TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

with tf.device('/gpu:0'):
    batch_size_ = 256
    epochs_ = 4
#     class_weight_ = get_class_weights(Y, smooth_factor=0.1)
    history = model.fit(X_t,y,batch_size=batch_size_,epochs=epochs_,validation_split=0.2, callbacks=[tensorBoardCB])

Train on 127656 samples, validate on 31915 samples
Epoch 1/4
127656/127656 [==============================] - 213s 2ms/step - loss: 0.0831 - auc: 0.8498 - val_loss: 0.0489 - val_auc: 0.9517
Epoch 2/4
127656/127656 [==============================] - 210s 2ms/step - loss: 0.0468 - auc: 0.9640 - val_loss: 0.0458 - val_auc: 0.9703
Epoch 3/4
127656/127656 [==============================] - 210s 2ms/step - loss: 0.0420 - auc: 0.9740 - val_loss: 0.0464 - val_auc: 0.9762
Epoch 4/4
127656/127656 [==============================] - 206s 2ms/step - loss: 0.0380 - auc: 0.9780 - val_loss: 0.0468 - val_auc: 0.9794


In [ ]:
pyplot.plot(history.history['acc'])
pyplot.show()

In [40]:
# test = pd.read_csv('test_clean.csv')
# test.comment_text = test.comment_text.astype(str)
# list_sentences_test = test["comment_text"]
# list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
# X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
# y_test = model.predict([X_te], batch_size=512, verbose=1)
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission.csv', index=False)